🟦 Cell 0 — Project Introduction

# 🎬 Movie Review Sentiment Analysis  
### Using Multilingual Translation + Transformers (English, French, Spanish)

This project analyzes the sentiment of movie reviews written in **three different languages**:

- 🇬🇧 English  
- 🇫🇷 French  
- 🇪🇸 Spanish  

We perform three major tasks:

---

## 🔹 1. Create a Unified Dataset  
We combine movie reviews from three CSV files into a **single Pandas DataFrame** containing:

- Title  
- Year  
- Synopsis  
- Review  
- Original Language  

French and Spanish files use different column names, so we normalize them into a consistent schema.

---

## 🔹 2. Translate Non-English Text  
We use **HuggingFace pretrained translation models**:

- `Helsinki-NLP/opus-mt-fr-en` (French → English)  
- `Helsinki-NLP/opus-mt-es-en` (Spanish → English)  

Both **Synopsis** and **Review** are translated so that all text is aligned in English.

---

## 🔹 3. Sentiment Analysis  
We use the transformer model:

- `distilbert-base-uncased-finetuned-sst-2-english`

It classifies each review as:

- **Positive**  
- **Negative**

A new column named **Sentiment** is added to the final DataFrame.

---

## 🔹 Final Output  
The notebook exports a CSV file named:



final_movie_sentiment.csv


This file contains all 30 movies with:

- Titles  
- Years  
- Translated synopses  
- Translated reviews  
- Original languages  
- Sentiment predictions  

---

## 🧠 Skills Demonstrated

- Pandas data cleaning  
- HuggingFace Transformers (translation + sentiment analysis)  
- PyTorch inference  
- Working with multilingual NLP  
- Creating reproducible research notebooks  

---

Let's begin! 👇

🔵 Cell 1 — Install & Import Packages

In [1]:
!pip install transformers sentencepiece

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification


🔵 Cell 2 — Load CSV Files From GitHub

In [2]:
url_eng = "https://raw.githubusercontent.com/saadiqbal96/movie-sentiment-analysis/main/data/movie_reviews_eng.csv"
url_fr  = "https://raw.githubusercontent.com/saadiqbal96/movie-sentiment-analysis/main/data/movie_reviews_fr.csv"
url_sp  = "https://raw.githubusercontent.com/saadiqbal96/movie-sentiment-analysis/main/data/movie_reviews_sp.csv"

df_eng = pd.read_csv(url_eng)
df_fr = pd.read_csv(url_fr)
df_sp = pd.read_csv(url_sp)


🔵 Cell 3 — preprocess_data()

In [5]:
def preprocess_data():
    # Rename English columns
    df_eng_renamed = df_eng.rename(columns={
        "Movie / TV Series": "Title",
        "Year": "Year",
        "Synopsis": "Synopsis",
        "Review": "Review",
        "Original Language": "Original Language"
    })

    # Rename French columns
    df_fr_renamed = df_fr.rename(columns={
        "Titre": "Title",
        "Année": "Year",
        "Synopsis": "Synopsis",
        "Critiques": "Review",
        "Original Language": "Original Language"
    })

    # Rename Spanish columns
    df_sp_renamed = df_sp.rename(columns={
        "Título": "Title",
        "Año": "Year",
        "Sinopsis": "Synopsis",
        "Críticas": "Review",
        "Original Language": "Original Language"
    })

    # Combine
    df = pd.concat([df_eng_renamed, df_fr_renamed, df_sp_renamed], ignore_index=True)
    return df[["Title", "Year", "Synopsis", "Review", "Original Language"]]

df = preprocess_data()
df.head()



Title  Year  \
0  The Shawshank Redemption   1994   
1           The Dark Knight   2008   
2               Forrest Gump  1994   
3             The Godfather   1972   
4                  Inception  2010   

                                            Synopsis  \
0  Andy Dufresne (Tim Robbins), a successful bank...   
1  Batman (Christian Bale) teams up with District...   
2  Forrest Gump (Tom Hanks) is a simple man with ...   
3  Don Vito Corleone (Marlon Brando) is the head ...   
4  Dom Cobb (Leonardo DiCaprio) is a skilled thie...   

                                              Review Original Language  
0  "The Shawshank Redemption is an inspiring tale...                en  
1  "The Dark Knight is a thrilling and intense su...                en  
2  "Forrest Gump is a heartwarming and inspiratio...                en  
3  "The Godfather is a classic movie that stands ...                en  
4  "Inception is a mind-bending and visually stun...                en

🔵 Cell 4 — Load Translation Models

In [6]:
tokenizer_fr = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
model_fr = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

tokenizer_sp = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
model_sp = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

🔵 Cell 5 — translate() Function

In [8]:
def translate(text, tokenizer, model):
    if pd.isna(text):
        return ""
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


🔵 Cell 6 — Translate Non-English

In [9]:
def translate_non_english(df):
    mask_fr = df["Original Language"] == "fr"
    mask_sp = df["Original Language"] == "sp"

    df.loc[mask_fr, "Review"] = df.loc[mask_fr, "Review"].apply(lambda x: translate(x, tokenizer_fr, model_fr))
    df.loc[mask_fr, "Synopsis"] = df.loc[mask_fr, "Synopsis"].apply(lambda x: translate(x, tokenizer_fr, model_fr))

    df.loc[mask_sp, "Review"] = df.loc[mask_sp, "Review"].apply(lambda x: translate(x, tokenizer_sp, model_sp))
    df.loc[mask_sp, "Synopsis"] = df.loc[mask_sp, "Synopsis"].apply(lambda x: translate(x, tokenizer_sp, model_sp))

    return df

df = translate_non_english(df)
df.head()


Title  Year  \
0  The Shawshank Redemption   1994   
1           The Dark Knight   2008   
2               Forrest Gump  1994   
3             The Godfather   1972   
4                  Inception  2010   

                                            Synopsis  \
0  Andy Dufresne (Tim Robbins), a successful bank...   
1  Batman (Christian Bale) teams up with District...   
2  Forrest Gump (Tom Hanks) is a simple man with ...   
3  Don Vito Corleone (Marlon Brando) is the head ...   
4  Dom Cobb (Leonardo DiCaprio) is a skilled thie...   

                                              Review Original Language  
0  "The Shawshank Redemption is an inspiring tale...                en  
1  "The Dark Knight is a thrilling and intense su...                en  
2  "Forrest Gump is a heartwarming and inspiratio...                en  
3  "The Godfather is a classic movie that stands ...                en  
4  "Inception is a mind-bending and visually stun...                en

🔵 Cell 7 — Load Sentiment Model

In [10]:
tokenizer_sent = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model_sent = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

🔵 Cell 8 — analyze_sentiment()

In [11]:
def analyze_sentiment(text):
    inputs = tokenizer_sent(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model_sent(**inputs).logits
    pred = torch.argmax(logits).item()
    return "Positive" if pred == 1 else "Negative"


🔵 Cell 9 — Apply Sentiment Analysis

In [12]:
df["Sentiment"] = df["Review"].apply(analyze_sentiment)
df.head()


Title  Year  \
0  The Shawshank Redemption   1994   
1           The Dark Knight   2008   
2               Forrest Gump  1994   
3             The Godfather   1972   
4                  Inception  2010   

                                            Synopsis  \
0  Andy Dufresne (Tim Robbins), a successful bank...   
1  Batman (Christian Bale) teams up with District...   
2  Forrest Gump (Tom Hanks) is a simple man with ...   
3  Don Vito Corleone (Marlon Brando) is the head ...   
4  Dom Cobb (Leonardo DiCaprio) is a skilled thie...   

                                              Review Original Language  \
0  "The Shawshank Redemption is an inspiring tale...                en   
1  "The Dark Knight is a thrilling and intense su...                en   
2  "Forrest Gump is a heartwarming and inspiratio...                en   
3  "The Godfather is a classic movie that stands ...                en   
4  "Inception is a mind-bending and visually stun...                en   

  Sentiment  
0  Positive  
1  Positive  
2  Positive  
3  Positive  
4  Positive

🔵 Cell 10 — Save Output CSV

In [13]:
output_path = "final_movie_sentiment.csv"
df.to_csv(output_path, index=False)

output_path


'final_movie_sentiment.csv'

🔵 Cell 11 — Download the CSV (optional)

In [15]:
from google.colab import files
files.download("/content/final_movie_sentiment.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🟦 Final Cell — Project Conclusion

# 🎉 Conclusion

In this project, we successfully analyzed multilingual movie reviews by integrating:

- **Data preprocessing**  
- **Machine translation**  
- **Sentiment analysis using Transformers**

We unified 30 movies across English, French, and Spanish, translated all non-English text into English, and predicted sentiment for each review.  
The final dataset is ready for further visualization, modeling, or reporting.

---

## 🚀 What We Achieved

✔ Combined three multilingual datasets  
✔ Translated reviews & synopses using pretrained Neural Machine Translation  
✔ Classified sentiment using DistilBERT  
✔ Produced a clean final CSV with 30 rows  
✔ Demonstrated end-to-end NLP workflow  

---

## 🙌 Thank You for Reviewing This Project  
The workflow aligns with modern industry NLP pipelines and demonstrates practical use of HuggingFace transformers for multiling